In [3]:
from time import timezone

import openai
import os
from typing import List
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from postgres_db.models.Article import Article

In [4]:
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

In [21]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
import dateutil.parser

scraping_delay = 100000
pages = ['https://www.stopfake.org/uk/category/context-ua/', 'https://www.stopfake.org/uk/category/context-ua/page/2/', 'https://www.stopfake.org/uk/category/context-ua/page/3/',
         'https://www.stopfake.org/uk/category/context-ua/page/4/', 'https://www.stopfake.org/uk/category/context-ua/page/5/', 'https://www.stopfake.org/uk/category/context-ua/page/6/']
article_data = []
for page_url in pages:
    page = requests.get(page_url)
    soup = BeautifulSoup(page.content, "html.parser")
    feed_list = soup.find("div", class_='td-ss-main-content')
    articles = feed_list.find_all("div", class_='td_module_10 td_module_wrap td-animation-stack')
    for article in articles:
        href = article.find('a')['href']
        timestamp = article.find('time', class_='entry-date updated td-module-date')['datetime']
        article_page = requests.get(href)
        soup = BeautifulSoup(article_page.content, "html.parser")
        title_tag = soup.find('h1', class_='entry-title')
        title = title_tag.get_text().replace('\xa0', ' ').strip()
        paragraphs_list = []
        paragraph_content = soup.find('div', class_='td-post-content tagdiv-type')
        paragraphs = paragraph_content.find_all('p')
        sources = []
        for paragraph in paragraphs:
            text = paragraph.get_text()
            text = text.replace(' ', ' ')
            text = text.replace('\xa0', ' ')
            paragraphs_list.append(text)
            links = paragraph.find_all('a')
            for link in links:
                source_url = link['href']
                if source_url:
                    sources.append(source_url)
        if href is not None and timestamp is not None and title is not None and paragraphs_list is not None and sources is not None:
            article_time = dateutil.parser.isoparse(timestamp)
            current_time = datetime.now(timezone.utc)
            one_hour_ago = current_time - timedelta(hours=scraping_delay)
            if article_time >= one_hour_ago:
                article_data.append({
                    'outlet': 'stopfake',
                    'href': href,
                    'timestamp': timestamp,
                    'title': title,
                    'paragraphs': paragraphs_list,
                    'sources': sources
                })
            else:
                break


In [22]:
len(article_data)

60

In [50]:
system_instruction = '''
Ти - асистент з агрегації новин незалежних організацій факт-чекерів.
Твоя задача - це розглянути статтю і надати їй категорію. 
Список категорій поки що формується, тому для кожної статті знайди центральну тему і опиши 1-2 словами.
Статті, які ти описуєш - це статті з викривання фейків, які були створені за час російскього вторгення в Україну починаючи з 2022р.
Тобі потрібно категоризувати сам фейк, тобто суб'єкт дезинформації. Наприклад, якщо в статті йдеться про фейк про російський ракетний удар по цивільному об'єкту, де 
фейк - це те, що в цивільному об'єкті насправді були військові,  о  
Ти маєш відповісти тільки категорією, і нічим більше. 
'''

In [51]:
from pydantic import BaseModel

class FindCategorySchema(BaseModel):
    category: str

In [52]:
import openai

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

assistant = client.beta.assistants.create(
        model='gpt-4.1-mini',
        instructions=system_instruction,
        temperature=0.3,
        response_format={
            'type': 'json_schema',
            'json_schema':
                {
                    'name': 'MetricSchema',
                    'schema': FindCategorySchema.model_json_schema()
                }
        }
    )

In [53]:
categories = []
for article in article_data:    
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": json.dumps(article),
            }
        ]
    )

    with client.beta.threads.runs.stream(
            thread_id=thread.id,
            assistant_id=assistant.id,
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(thread_id=thread.id)
    result = messages.data[0].content[0].text.value
    result = json.loads(result)
    categories.append(result['category'])
    client.beta.threads.delete(thread_id=thread.id)


['дезінформація про міжнародні відносини та політику',
 'маніпуляція щодо символіки та історії Дня Європи',
 'оманливі заяви про туристичний потік у Росії',
 'оманливі заяви про звільнення військових',
 'створення недовіри до біженців',
 'оманливе виправдання ракетного удару',
 'маніпуляції щодо мирних ініціатив Трампа',
 'маніпуляції з релігією та церквою',
 'оманливе виправдання атак на цивільні судна',
 'оманливе виправдання економічних витрат ЄС на Україну',
 'маніпуляції з міжнародним правом для виправдання агресії',
 'російська пропаганда та дезінформація',
 'маніпуляція інформацією про умови життя в Європі',
 'російська пропаганда у США',
 'маніпуляції з жертвами серед дітей',
 'маніпуляція громадською думкою',
 "оманливе виправдання атаки по цивільному об'єкту",
 'пропаганда Кремля',
 'легітимізація російської влади в Україні',
 'оманливе виправдання вбивства журналістів',
 'дискредитація українських військових і влади',
 'російська пропаганда та дезінформація',
 'оманливе запе